In [212]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [294]:
import betfairlightweight
import flumine
from flumine.resources import recorder
from flumine.storage import storageengine
import json
from datetime import datetime as dt
import datetime
import h2o

In [295]:
from agent import Agent

In [296]:
model = './models/GLM_grid_1_AutoML_20200104_075510_model_1/GLM_grid_1_AutoML_20200104_075510_model_1.zip'

In [297]:
run_time = 19
mrkt = '1.167076164'
market_ids = [mrkt]

In [298]:
end = dt.now() + datetime.timedelta(minutes=run_time)
end.strftime("%H:%M:%S")

'17:45:20'

In [299]:
market_filter = {"marketIds": market_ids}
storage_engine = storageengine.Local('./data')

rcd =  recorder.MarketRecorder(storage_engine, market_filter=market_filter)

In [300]:
with open('secrets/config.json', 'r') as fp:
    config = json.load(fp)

trading = betfairlightweight.APIClient(**config)
trading.login()

<LoginResource>

In [247]:
bot = Agent(mrkt, trading, rcd, model)

In [248]:
bot.init_stream()

In [250]:
bot.flumine_object.status

'running'

In [286]:
bot.poll_directory()
bot_df = bot.market_df()
bot_df[bot_df.sid==bot.sid].shape

(50, 18)

In [292]:
bot.assess_strategy(verbose=True)

0.1954616710107983


In [288]:
print(bot.current_direction)

None


In [289]:
bot.trade_price

In [290]:
bot.in_market

False

In [293]:
while dt.now() < end:
    bot.poll_directory()
    if not bot.in_market:
        bot.assess_strategy(verbose=True)
    bot.manage_trade()
    
print('END END END')

190670495749_sl
{'limitOrder': {'size': 0.03, 'persistenceType': 'LAPSE', 'price': 50.0}, 'side': 'LAY', 'selectionId': 23405261, 'orderType': 'LIMIT'}
SUCCESS 190671230131
0.1947426547712266
1.167076164_bot
{'limitOrder': {'size': 0.03, 'persistenceType': 'LAPSE', 'price': 5.1}, 'side': 'BACK', 'selectionId': 23405261, 'orderType': 'LIMIT'}
SUCCESS 190671234076
190671234076_tp
{'limitOrder': {'size': 0.03, 'persistenceType': 'LAPSE', 'price': 4.7}, 'side': 'LAY', 'selectionId': 23405261, 'orderType': 'LIMIT'}
SUCCESS 190671238556
190670232135_sl
{'limitOrder': {'size': 0.03, 'persistenceType': 'LAPSE', 'price': 50.0}, 'side': 'LAY', 'selectionId': 23405261, 'orderType': 'LIMIT'}
SUCCESS 190671248706
0.2538466166573385
0.2520123920744744
0.24965297006164086
1.167076164_bot
{'limitOrder': {'size': 0.03, 'persistenceType': 'LAPSE', 'price': 5.4}, 'side': 'BACK', 'selectionId': 23405261, 'orderType': 'LIMIT'}
SUCCESS 190671253788
190671234076_tp
{'limitOrder': {'size': 0.03, 'persistenceT

ERROR:flumine.flumine:Betfair error: [Connect: 1002.0]: Socket The read operation timed out
Traceback (most recent call last):
  File "/Users/jake/miniconda3/envs/betfair/lib/python3.6/site-packages/betfairlightweight/streaming/betfairstream.py", line 212, in _receive_all
    part = self._socket.recv(self.buffer_size)
  File "/Users/jake/miniconda3/envs/betfair/lib/python3.6/ssl.py", line 997, in recv
    return self.read(buflen)
  File "/Users/jake/miniconda3/envs/betfair/lib/python3.6/ssl.py", line 874, in read
    return self._sslobj.read(len, buffer)
  File "/Users/jake/miniconda3/envs/betfair/lib/python3.6/ssl.py", line 633, in read
    v = self._sslobj.read(len)
socket.timeout: The read operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jake/miniconda3/envs/betfair/lib/python3.6/site-packages/flumine/flumine.py", line 122, in _run
    self._socket.start(async_=False)
  File "/Users/jake/minic

In [ ]:
bot.stop()

In [241]:
trading.logout()

<LogoutResource>